In [1]:
import os
import torch
from torch import nn
from tqdm.notebook import tqdm
from torchvision import transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
from torch.optim import Adam
import matplotlib.pyplot as plt
import pandas as pd

## Generator class

In [2]:
class Generator(nn.Module):
    def __init__(self, z_dim=10, pred_dim=4, hidden_dim=32):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            
            nn.Linear(z_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.LeakyReLU(inplace=True),
            
            ### To complete
            nn.Linear(hidden_dim, hidden_dim*2),
            nn.BatchNorm1d(hidden_dim*2),
            nn.LeakyReLU(),

            nn.Linear(2*hidden_dim, 4*hidden_dim),
            nn.BatchNorm1d(4*hidden_dim),
            nn.LeakyReLU(),

            nn.Linear(4*hidden_dim, 2*hidden_dim),
            nn.BatchNorm1d(2*hidden_dim),
            nn.LeakyReLU(),
            ###
            nn.Linear(2 * hidden_dim, pred_dim),
            nn.Sigmoid()
        )
    def forward(self, noise):
        return self.gen(noise)

Creates a gaussian noise sample to input in the generator

In [3]:
def get_noise(n_samples, z_dim, device=torch.device('cpu')):
    ### To complete
    noise = torch.randn((n_samples, z_dim), device=device)
    return noise
    ###

## Discriminator class

In [4]:
class Discriminator(nn.Module):

    def __init__(self, pred_dim=4, hidden_dim=32):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            ### To complete
            nn.Linear(pred_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(hidden_dim, 2*hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Linear(2*hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
            ###
        )

    def forward(self, image):
        return self.disc(image)

## Training

In [5]:
# Setting architecture
z_dim = 10
pred_dim = 4
hidden_dim=32

# Setting optimization hyperparameters
n_epochs = 100
batch_size = 128
lr = 1e-4
criterion = nn.BCEWithLogitsLoss()
device = torch.device('cpu')

dataloader = DataLoader(
    dataset=pd.read_csv("data/station_40.csv"), 
    batch_size=batch_size,
    shuffle=True,
    drop_last=True)

### Initialization

Initialization of the generator and the discriminator

In [6]:
# Initialization of the generator and discriminator
gen = Generator(z_dim,pred_dim,hidden_dim)
#gen.to(device)
disc = Discriminator(pred_dim,hidden_dim)
#disc.to(device)

# Initialization of their respective optimizers (Use Adam optimizer)
### To complete
gen_opt = torch.optim.Adam(gen.parameters(), lr)
disc_opt = torch.optim.Adam(disc.parameters(), lr)
###

print('Generator\n')
print(gen)

print('Discriminator\n')
print(disc)

Generator

Generator(
  (gen): Sequential(
    (0): Linear(in_features=10, out_features=32, bias=True)
    (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01, inplace=True)
    (3): Linear(in_features=32, out_features=64, bias=True)
    (4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): LeakyReLU(negative_slope=0.01)
    (6): Linear(in_features=64, out_features=128, bias=True)
    (7): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): LeakyReLU(negative_slope=0.01)
    (9): Linear(in_features=128, out_features=64, bias=True)
    (10): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): LeakyReLU(negative_slope=0.01)
    (12): Linear(in_features=64, out_features=4, bias=True)
    (13): Sigmoid()
  )
)
Discriminator

Discriminator(
  (disc): Sequential(
    (0): Linear(in_features=4, out_features=

## Discriminator loss

In [7]:
def get_disc_loss(gen, disc, criterion, real, num_images, z_dim, device):
    ### To complete
    # 1. Create noise vectors and generate a batch (num_images) of fake images. 
    noise = get_noise(num_images, z_dim, device)
    images = gen(noise)
        
    # 2.Get the discriminator's prediction of the fake image and calculate the loss. 
    pred = disc(images)
    loss_fake = criterion(pred, torch.zeros(num_images, 1).to(device))

    
    # 3. Get the discriminator's prediction of the real image and calculate the loss.
    real_pred = disc(real)
    loss_real = criterion(real_pred, torch.ones(real.size(dim=0), 1).to(device))
    
    # 4. Calculate the discriminator's loss by averaging the real and fake loss
    # and set it to disc_loss.
    #loss = torch.cat((loss_real, loss_fake), dim=0)
    disc_loss = (loss_fake + loss_real)/2
    ###
    return disc_loss

In [8]:
get_disc_loss(gen,disc,criterion, torch.randn((batch_size,pred_dim),device='cpu'),batch_size,z_dim,device='cpu')

tensor(0.6940, grad_fn=<DivBackward0>)

## Generator loss

In [9]:
def get_gen_loss(gen, disc, criterion, num_images, z_dim, device):
    ### To complete
    # 1. Create noise vectors and generate a batch (num_images) of fake images. 
    # Make sure to pass the device argument to the noise.
    noise = get_noise(num_images,z_dim,device)
    images = gen(noise)
        
    # 2.Get the discriminator's prediction of the fake image
    pred = disc(images)
    
    # 3. Calculate the generator's loss. 
    gen_loss = criterion(pred, torch.ones(num_images,1).to(device))
    return gen_loss

In [10]:
get_gen_loss(gen,disc,criterion,batch_size,z_dim,device)

tensor(0.6777, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

# Training

In [11]:
noise = get_noise(5,z_dim,device)
gen(noise)

tensor([[0.4184, 0.2737, 0.6896, 0.4274],
        [0.5529, 0.5940, 0.4609, 0.5258],
        [0.4347, 0.5536, 0.3948, 0.5962],
        [0.5837, 0.4873, 0.4688, 0.5042],
        [0.3131, 0.6005, 0.4665, 0.6165]], grad_fn=<SigmoidBackward0>)

In [17]:
for i, point in enumerate(pd.read_csv("data/station_40.csv")):
    print(i, point)

0 YEAR
1 W_1
2 W_2
3 W_3
4 W_4
5 W_5
6 W_6
7 W_7
8 W_8
9 W_9
10 W_10
11 W_11
12 W_12
13 W_13
14 W_14
15 W_15
16 W_16
17 W_17
18 W_18
19 YIELD


In [12]:
G_losses = []
D_losses = []

for epoch in range(1, n_epochs+1):
  
    # Dataloader returns the batches and tqdm is just a wrapper to add a progressbar
    # when iterating over the dataloader
    #dloader = tqdm(dataloader, unit='batches')
    # Initialize tracked metrics
    sum_generator_loss = 0
    sum_discriminator_loss = 0
    
    for batch_index, (real, _) in enumerate(dataloader, start=0):
        cur_batch_size = len(real)

        # Flatten the batch of real images from the dataset
        #real = real.view(cur_batch_size, -1).to(device)

        ##################### To complete
        
        ## 1. Discriminator training
        #    A. Zero out discriminator gradients
        disc_opt.zero_grad()

        #    B. Compute discriminator loss
        disc_loss = get_disc_loss(gen,disc,criterion,real,cur_batch_size,z_dim,device)

        #    C. Compute discriminator loss gradients with respect to discriminator params 
        disc_loss.backward()

        #    D. Update discriminator params
        disc_opt.step()
        
        
        ## 2. Generator training
        #    A. Zero out generator gradients
        gen_opt.zero_grad()
        
        #    B. Compute generator loss on fake images
        gen_loss = get_gen_loss(gen,disc,criterion,cur_batch_size,z_dim,device)

        #    C. Compute generator loss gradients with respect to generator params 
        gen_loss.backward()

        #    D. Update generator params
        gen_opt.step()
        
        #####################

        # Keep track of the average discriminator loss
        sum_discriminator_loss += disc_loss.item()
        mean_discriminator_loss = sum_discriminator_loss/batch_index

        # Keep track of the average generator loss
        sum_generator_loss += gen_loss.item()
        mean_generator_loss = sum_generator_loss/batch_index

        # Save Losses for plotting later
        G_losses.append(mean_generator_loss)
        D_losses.append(mean_discriminator_loss)
        
        # Update progress bar to see metrics during training
        desc = "Epoch {}/{}: gen loss: {:.3f}, disc loss: {:.3f}".format(epoch, n_epochs, mean_generator_loss, mean_discriminator_loss)
        dloader.set_description(desc)


# For last iteration plot real + fake images
gen.eval()
fake_noise = get_noise(cur_batch_size, z_dim, device=device)
fake = gen(fake_noise)


KeyError: 9947

In [ ]:
G_losses = []
D_losses = []

for epoch in range(1, n_epochs+1):
  
    # Dataloader returns the batches and tqdm is just a wrapper to add a progressbar
    # when iterating over the dataloader
    dloader = tqdm(dataloader, unit='batches')
    # Initialize tracked metrics
    sum_generator_loss = 0
    sum_discriminator_loss = 0
    
    for batch_index, (real, _) in enumerate(dloader, start=0):
        cur_batch_size = len(real)

        # Flatten the batch of real images from the dataset
        #real = real.view(cur_batch_size, -1).to(device)

        ##################### To complete
        
        ## 1. Discriminator training
        #    A. Zero out discriminator gradients
        disc_opt.zero_grad()

        #    B. Compute discriminator loss
        disc_loss = get_disc_loss(gen,disc,criterion,real,cur_batch_size,z_dim,device)

        #    C. Compute discriminator loss gradients with respect to discriminator params 
        disc_loss.backward()

        #    D. Update discriminator params
        disc_opt.step()
        
        
        ## 2. Generator training
        #    A. Zero out generator gradients
        gen_opt.zero_grad()
        
        #    B. Compute generator loss on fake images
        gen_loss = get_gen_loss(gen,disc,criterion,cur_batch_size,z_dim,device)

        #    C. Compute generator loss gradients with respect to generator params 
        gen_loss.backward()

        #    D. Update generator params
        gen_opt.step()
        
        #####################

        # Keep track of the average discriminator loss
        sum_discriminator_loss += disc_loss.item()
        mean_discriminator_loss = sum_discriminator_loss/batch_index

        # Keep track of the average generator loss
        sum_generator_loss += gen_loss.item()
        mean_generator_loss = sum_generator_loss/batch_index

        # Save Losses for plotting later
        G_losses.append(mean_generator_loss)
        D_losses.append(mean_discriminator_loss)
        
        # Update progress bar to see metrics during training
        desc = "Epoch {}/{}: gen loss: {:.3f}, disc loss: {:.3f}".format(epoch, n_epochs, mean_generator_loss, mean_discriminator_loss)
        dloader.set_description(desc)


# For last iteration plot real + fake images
gen.eval()
fake_noise = get_noise(cur_batch_size, z_dim, device=device)
fake = gen(fake_noise)


  0%|          | 0/78 [00:00<?, ?batches/s]

KeyError: 3755